# Binoxxo Puzzle

Lucerne University of Applied Sciences and Arts - School of Information Technology

Place X or O in the empty cells so that there are no more than two consecutive X's or O's in a row or a column.
The number of X's is the same as the number of O's in each row and column, and all rows and columns are unique.

Find more Binoxxo puzzles [here](https://www.binoxxo.ch/Binoxxo-Raetselbuch/)

In [1]:
%pip install ortools

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from ortools.sat.python import cp_model
from itertools import product

Binoxxo puzzle from lecture

In [3]:
binoxxo1 = [
    ["", "1", "", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["", "1", "1", "", "", "", "", "", "", ""],
    ["", "", "", "", "0", "0", "", "", "", "0"],
    ["1", "", "", "", "", "", "1", "1", "", ""],
    ["", "1", "", "", "1", "", "", "", "", ""],
    ["", "", "", "0", "", "", "1", "", "", ""],
    ["", "0", "", "", "", "", "", "0", "", "0"],
    ["", "", "", "", "0", "", "", "", "", ""],
    ["0", "", "", "", "", "", "", "", "", "0"]
]

And two more examples

In [ ]:
binoxxo2 = [
    ["", "", "0", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["0", "", "", "", "1", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", ""],
    ["0", "", "", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "", "", "", "", "", "1", "1", ""],
    ["1", "", "", "", "", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", "0"],
    ["", "", "", "", "1", "", "0", "", "", ""]
]
binoxxo3 = [
    ["1", "", "", "", "", "", "0", "0", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "0", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "1", "", "", "", "0"],
    ["1", "", "1", "", "", "1", "", "", "1", ""],
    ["", "", "1", "", "1", "", "", "", "", ""],
    ["", "", "", "", "1", "1", "", "", "1", ""],
    ["", "", "", "", "", "", "", "", "", "0"],
    ["", "1", "", "1", "", "", "", "", "", ""],
    ["", "1", "", "", "1", "", "", "1", "1", ""]
]

Pick one of the examples

In [4]:
game = binoxxo1

Create model

In [5]:
model = cp_model.CpModel()

In [6]:
# Type your model here ...
boardSize = len(game)

sameAmountOfXsAndOs = boardSize // 2

board = [[model.NewIntVar(0, 1, f"({i},{j})") for j in range(boardSize)] for i in range(boardSize)]

# rows can't be the same
# model.AddAllDifferent([board[i] for i in range(boardSize)])

#columns can't be the same, compare strings of full column
for i in range(boardSize):
  model.AddAllDifferent([int("".join(str(board[j][i] for j in range(boardSize))))])

#set values for selected game
for i in range(boardSize):
    for j in range(boardSize):
        if game[i][j] in {"0", "1"}:
            model.Add(board[i][j] == int(game[i][j]))

for i in range(boardSize):
  # no more than 2 Xs in a row (X is represented by 1, so sum of three following fields should never be 3)
  model.Add(cp_model.LinearExpr.Sum(board[i][j], board[i][j+1], board[i][j+2]) < 3 for j in range(boardSize-2))
  # no more than 2 Os in a row (O is represented by 0, so sum of three following fields should be greater than 0)
  model.Add(cp_model.LinearExpr.Sum(board[i][j], board[i][j+1], board[i][j+2]) > 0 for j in range(boardSize-2))
  # no more than 2 Xs in a column
  model.Add(cp_model.LinearExpr.Sum(board[j][i], board[j+1][i], board[j+2][i]) < 3 for j in range(boardSize-2))
  # no more than 2 Os in a column
  model.Add(cp_model.LinearExpr.Sum(board[j][i], board[j+1][i], board[j+2][i]) > 0 for j in range(boardSize-2))

# every column and row sum should equal sameAmountOfXsAndOs as O is represented as 0 (zero) and thus sum is exactly the amount of Xs
for i in range(boardSize):
    model.Add(cp_model.LinearExpr.Sum([board[i][j] for j in range(boardSize)]) == sameAmountOfXsAndOs)
    model.Add(cp_model.LinearExpr.Sum([board[j][i] for j in range(boardSize)]) == sameAmountOfXsAndOs)

ValueError: invalid literal for int() with base 10: '<generator object <genexpr> at 0x000001E2618264D0>'

Callback for solution printing

In [ ]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables

    def on_solution_callback(self):
        mapper = {0:'O', 1:'X'}
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{mapper[self.Value(self.__variables[i][j])]}] ", end='')
            print("\n")
        print("\n")

Solve and print all solutions

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, SolutionPrinter(board))

In [ ]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")